# 一、概述¶
## 1. matplotlib的三层api
matplotlib的原理或者说基础逻辑是，用Artist对象在画布(canvas)上绘制(Render)图形。
就和人作画的步骤类似：

准备一块画布或画纸
准备好颜料、画笔等制图工具
作画
所以matplotlib有三个层次的API：

matplotlib.backend_bases.FigureCanvas 代表了绘图区，所有的图像都是在绘图区完成的
matplotlib.backend_bases.Renderer 代表了渲染器，可以近似理解为画笔，控制如何在 FigureCanvas 上画图。
matplotlib.artist.Artist 代表了具体的图表组件，即调用了Renderer的接口在Canvas上作图。
前两者处理程序和计算机的底层交互的事项，第三项Artist就是具体的调用接口来做出我们想要的图，比如图形、文本、线条的设定。所以通常来说，我们95%的时间，都是用来和matplotlib.artist.Artist类打交道的。


## 2. Artist的分类
Artist有两种类型：primitives 和containers。
primitive是基本要素，它包含一些我们要在绘图区作图用到的标准图形对象，如曲线Line2D，文字text，矩形Rectangle，图像image等。
container是容器，即用来装基本要素的地方，包括图形figure、坐标系Axes和坐标轴Axis。他们之间的关系如下图所示：

## 3. matplotlib标准用法
matplotlib的标准使用流程为：

1. 创建一个Figure实例
1. 使用Figure实例创建一个或者多个Axes或Subplot实例
1. 使用Axes实例的辅助方法来创建primitive
值得一提的是，Axes是一种容器，它可能是matplotlib API中最重要的类，并且我们大多数时间都花在和它打交道上。更具体的信息会在之后容器小节说明。

一个流程示例及说明如下：

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# step 1 
# 我们用 matplotlib.pyplot.figure() 创建了一个Figure实例
fig = plt.figure()

# step 2
# 然后用Figure实例创建了一个两行一列(即可以有两个subplot)的绘图区，并同时在第一个位置创建了一个subplot
ax = fig.add_subplot(2, 1, 1) # two rows, one column, first plot

# step 3
# 然后用Axes实例的方法画了一条曲线
t = np.arange(0.0, 1.0, 0.01)
s = np.sin(2*np.pi*t)
line, = ax.plot(t, s, color='blue', lw=2)

# 二、自定义你的Artist对象¶
## 1. Artist属性
在图形中的每一个元素都对应着一个matplotlib Artist，且都有其对应的配置属性列表。

Figure本身包含一个Rectangle，Rectangle的大小就是Figure的大小；你可以用来设置Figure的背景色和透明度。
每个Axes边界框(默认白底黑边)，也有一个Rectangle，通过它可以设置Axes的颜色、透明度等。
这些实例都存储在成员变量(member variables) Figure.patch 和 Axes.patch中。 （Patch是一个来源于MATLAB的名词，它是图形上颜色的一个2D补丁，包含rectangels-矩形，circles-圆 和 plygons-多边形）

换个表达方式：
Figure.patch属性：是一个Rectangle，代表了图表的矩形框，它的大小就是图表的大小， 并且可以通过它设置figure的背景色和透明度。
Axes.patch属性：也是一个Rectangle，代表了绘图坐标轴内部的矩形框（白底黑边）， 通过它可以设置Axes的颜色、透明度等。

每个matplotlib Artist都有以下属性：

.alpha属性：透明度。值为0—1之间的浮点数
.axes属性：返回这个Artist所属的axes，可能为None
.figure属性：该Artist所属的Figure，可能为None
.label：一个text label
.visible：布尔值，控制Artist是否绘制
这里仅列举几个常见的属性，更详细的属性清单请查阅官方文档： Artist属性列表

In [ ]:
# .patch
plt.figure().patch
plt.axes().patch

## 2. 属性调用方式
Artist对象的所有属性都通过相应的 get_* 和 set_* 函数进行读写。
例如下面的语句将alpha属性设置为当前值的一半：

a = o.get_alpha()
o.set_alpha(0.5*a)

如果想一次设置多个属性，也可以用set方法：

o.set(alpha=0.5, zorder=2)

可以使用 matplotlib.artist.getp(o,"alpha") 来获取属性，如果指定属性名，则返回对象的该属性值；如果不指定属性名，则返回对象的所有的属性和值。

In [ ]:
import matplotlib
# Figure rectangle的属性
matplotlib.artist.getp(fig.patch)

# 三、基本元素 - primitives
现在我们知道了如何检查和设置给定对象的属性，我们还需要知道如何获取该对象。

前文介绍到，Artist包含两种对象：基本要素-primitives 和 容器-containers。
primitives是基本要素，它包含一些我们要在绘图区作图用到的标准图形对象，如曲线Line2D，文本text，矩形Rectangle，图像image等。
container是容器，即用来装基本要素的地方，包括图形figure、坐标系Axes和坐标轴Axis。

本章重点介绍下 primitives 的几种类型：曲线-Line2D，矩形-Rectangle，图像-image

## 3.1. 2DLines
在matplotlib中曲线的绘制，主要是通过类 matplotlib.lines.Line2D 来完成的。
它的基类: matplotlib.artist.Artist

matplotlib中线-line的含义：它表示的可以是连接所有顶点的实线样式，也可以是每个顶点的标记。此外，这条线也会受到绘画风格的影响，比如，我们可以创建虚线种类的线。

它的构造函数：

```
class matplotlib.lines.Line2D(xdata, ydata, linewidth=None, linestyle=None, color=None, marker=None, markersize=None, markeredgewidth=None, markeredgecolor=None, markerfacecolor=None, markerfacecoloralt='none', fillstyle=None, antialiased=None, dash_capstyle=None, solid_capstyle=None, dash_joinstyle=None, solid_joinstyle=None, pickradius=5, drawstyle=None, markevery=None, **kwargs
```

其中常用的的参数有：

xdata:需要绘制的line中点的在x轴上的取值，若忽略，则默认为range(1,len(ydata)+1)
ydata:需要绘制的line中点的在y轴上的取值
linewidth:线条的宽度
linestyle:线型
color:线条的颜色
marker:点的标记，详细可参考markers API
markersize:标记的size
其他详细参数可参考Line2D官方文档

### a. 如何设置Line2D的属性
有三种方法可以用设置线的属性。

#### 1) 直接在plot()函数中设置


In [ ]:
import matplotlib.pyplot as plt
x = range(0,5)
y = [2,5,7,8,10]
plt.plot(x,y, linewidth=10) # 设置线的粗细参数为10

#### 2) 通过获得线对象，对线对象进行设置

In [ ]:
x = range(0,5)
y = [2,5,7,8,10]
line, = plt.plot(x, y, '-')
line.set_antialiased(False) # 关闭抗锯齿功能

#### 3) 获得线属性，使用setp()函数设置



In [ ]:
x = range(0,5)
y = [2,5,7,8,10]
lines = plt.plot(x, y)
plt.setp(lines, color='r', linewidth=10)

### b. 如何绘制lines¶
#### 1) 绘制直线line 常用的方法有两种

pyplot方法绘制
Line2D对象绘制

##### 1. pyplot方法绘制

In [ ]:
import matplotlib.pyplot as plt
x = range(0,5)
y = [2,5,7,8,10]
plt.plot(x,y)

##### 2. Line2D对象绘制

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

fig = plt.figure()
ax = fig.add_subplot(111)
line = Line2D(x, y)
ax.add_line(line)
ax.set_xlim(min(x), max(x))
ax.set_ylim(min(y), max(y))

plt.show()

##### 2) errorbar绘制误差折线图
pyplot里有个专门绘制误差线的功能，通过errorbar类实现，它的构造函数：
```
matplotlib.pyplot.errorbar(x, y, yerr=None, xerr=None, fmt='', ecolor=None, elinewidth=None, capsize=None, barsabove=False, lolims=False, uplims=False, xlolims=False, xuplims=False, errorevery=1, capthick=None, *, data=None, **kwargs)
```
其中最主要的参数是前几个:

x：需要绘制的line中点的在x轴上的取值
y：需要绘制的line中点的在y轴上的取值
yerr：指定y轴水平的误差
xerr：指定x轴水平的误差
fmt：指定折线图中某个点的颜色，形状，线条风格，例如‘co--’
ecolor：指定error bar的颜色
elinewidth：指定error bar的线条宽度
绘制errorbar



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure()
x = np.arange(10)
y = 2.5 * np.sin(x / 20 * np.pi)
yerr = np.linspace(0.05, 0.2, 10)
plt.errorbar(x, y + 3, yerr=yerr, label='both limits (default)')

## 3.2. patches
matplotlib.patches.Patch类是二维图形类。它的基类是matplotlib.artist.Artist，它的构造函数：
详细清单见 matplotlib.patches API
```
Patch(edgecolor=None, facecolor=None, color=None, linewidth=None, linestyle=None, antialiased=None, hatch=None, fill=True, capstyle=None, joinstyle=None, **kwargs)
```
### a. Rectangle-矩形
Rectangle矩形类在官网中的定义是： 通过锚点xy及其宽度和高度生成。 Rectangle本身的主要比较简单，即xy控制锚点，width和height分别控制宽和高。它的构造函数：

class matplotlib.patches.Rectangle(xy, width, height, angle=0.0, **kwargs)

在实际中最常见的矩形图是hist直方图和bar条形图。

#### 1) hist-直方图

```
matplotlib.pyplot.hist(x,bins=None,range=None, density=None, bottom=None, histtype='bar', align='mid', log=False, color=None, label=None, stacked=False, normed=None)
```

下面是一些常用的参数：

1. x: 数据集，最终的直方图将对数据集进行统计
1. bins: 统计的区间分布
1. range: tuple, 显示的区间，range在没有给出bins时生效
1. density: bool，默认为false，显示的是频数统计结果，为True则显示频率统计结果，这里需要注意，频率统计结果=区间数目/(总数*区间宽度)，和normed效果一致，官方推荐使用density
1. histtype: 可选{'bar', 'barstacked', 'step', 'stepfilled'}之一，默认为bar，推荐使用默认配置，step使用的是梯状，stepfilled则会对梯状内部进行填充，效果与bar类似
1. align: 可选{'left', 'mid', 'right'}之一，默认为'mid'，控制柱状图的水平分布，left或者right，会有部分空白区域，推荐使用默认
1. log: bool，默认False,即y坐标轴是否选择指数刻度
1. stacked: bool，默认为False，是否为堆积状图

hist绘制直方图

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x=np.random.randint(0,100,100) #生成【0-100】之间的100个数据,即 数据集
bins=np.arange(0,101,10) #设置连续的边界值，即直方图的分布区间[0,10],[10,20]...
plt.hist(x,bins,color='fuchsia',alpha=0.5)#alpha设置透明度，0为完全透明
plt.xlabel('scores')
plt.ylabel('count')
plt.xlim(0,100)#设置x轴分布范围 plt.show()

Rectangle矩形类绘制直方图

In [17]:
import pandas as pd
import re
df = pd.DataFrame(columns = ['data'])
df.loc[:,'data'] = x
df['fenzu'] = pd.cut(df['data'], bins=bins, right = False,include_lowest=True)

df_cnt = df['fenzu'].value_counts().reset_index()
df_cnt.loc[:,'mini'] = df_cnt['index'].astype(str).map(lambda x:re.findall('\[(.*)\,',x)[0]).astype(int)
df_cnt.loc[:,'maxi'] = df_cnt['index'].astype(str).map(lambda x:re.findall('\,(.*)\)',x)[0]).astype(int)
df_cnt.loc[:,'width'] = df_cnt['maxi']- df_cnt['mini']
df_cnt.sort_values('mini',ascending = True,inplace = True)
df_cnt.reset_index(inplace = True,drop = True)

#用Rectangle把hist绘制出来
import matplotlib.pyplot as plt

fig = plt.figure()
ax1 = fig.add_subplot(111)
#rect1 = plt.Rectangle((0,0),10,10)
#ax1.add_patch(rect)

#ax2 = fig.add_subplot(212)
for i in df_cnt.index:
    rect =  plt.Rectangle((df_cnt.loc[i,'mini'],0),df_cnt.loc[i,'width'],df_cnt.loc[i,'fenzu'])
    #rect2 = plt.Rectangle((10,0),10,5)
    ax1.add_patch(rect)
#ax1.add_patch(rect2)
ax1.set_xlim(0, 100)
ax1.set_ylim(0, 16)
plt.show()

<>:8: SyntaxWarning: invalid escape sequence '\['
<>:9: SyntaxWarning: invalid escape sequence '\,'
<>:8: SyntaxWarning: invalid escape sequence '\['
<>:9: SyntaxWarning: invalid escape sequence '\,'
/tmp/ipykernel_279375/331414219.py:8: SyntaxWarning: invalid escape sequence '\['
  df_cnt.loc[:,'mini'] = df_cnt['index'].astype(str).map(lambda x:re.findall('\[(.*)\,',x)[0]).astype(int)
/tmp/ipykernel_279375/331414219.py:9: SyntaxWarning: invalid escape sequence '\,'
  df_cnt.loc[:,'maxi'] = df_cnt['index'].astype(str).map(lambda x:re.findall('\,(.*)\)',x)[0]).astype(int)
/root/.cache/JetBrains/RemoteDev/dist/ea26855e7353e_pycharm-professional-2024.3.3/plugins/python-ce/helpers/pydev/_pydevd_bundle/pydevd_vars.py:725: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kind = df.dtypes[0].kind


KeyError: 'index'